<a href="https://colab.research.google.com/github/Yuuuugo/Food101/blob/main/Food101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi -L #Factory reset the runtime until it is set to a Tesla T4

GPU 0: Tesla T4 (UUID: GPU-0a19e429-b6f0-397e-67df-50fd14cc54cd)


In [7]:
import tensorflow as tf 
import tensorflow_datasets as tfds 
import matplotlib.pyplot as plt

In [4]:
# Import helper functions file
!wget https://raw.githubusercontent.com/Yuuuugo/Food101/main/helper_functions.py

--2022-04-18 14:31:14--  https://raw.githubusercontent.com/Yuuuugo/Food101/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10247 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-04-18 14:31:15 (76.3 MB/s) - ‘helper_functions.py’ saved [10247/10247]



In [5]:
from helper_functions import *

In [9]:
(train_data, test_data ) = tfds.load(name ="food101",
                                              split = ["train","validation"],
                                              shuffle_files = True,
                                              as_supervised = True,
                                              with_info = False)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteIC1AP4/food101-train.tfrecord


  0%|          | 0/75750 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteIC1AP4/food101-validation.tfrecord


  0%|          | 0/25250 [00:00<?, ? examples/s]

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.
